In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# !pip install --user --upgrade pip
# !pip install --user -q tensorflow==2.0.0-alpha0

import tensorflow as tf

# tf.enable_eager_execution()
print("Eager Execution: {}".format(tf.executing_eagerly()))
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution is: {}".format(tf.executing_eagerly()))
print("Keras version: {}".format(tf.keras.__version__))

# Install a temporary patch to enable a few extra TF 2.0 upgrades. This piece will be removed soon.

from tensorflow.python.ops import control_flow_util
control_flow_util.ENABLE_CONTROL_FLOW_V2 = True

#@tf.function
var = tf.Variable([3, 3])

if tf.test.is_gpu_available(): 
    print('Running on GPU')
    print('GPU #0?')
    print(var.device.endswith('GPU:0'))
else: 
    print('Running on CPU')

Eager Execution: True
TensorFlow version: 2.0.0-alpha0
Eager execution is: True
Keras version: 2.2.4-tf
Running on CPU


In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [4]:
# Normalizing Data

x_train, y_train = tf.cast(x_train/255., dtype=tf.float32), tf.cast(y_train/255., tf.float32)
x_test, y_test = tf.cast(x_test/255., dtype=tf.float32), tf.cast(y_test/255., tf.float32)

In [5]:
batch_size = 32
epochs = 10
num_classes = 10

In [6]:
# Using Dataset to represent an input Pipline

# https://www.tensorflow.org/guide/datasets#consuming_numpy_arrays
ds_train = tf.data.Dataset.from_tensor_slices( (x_train, y_train) ).batch(32).shuffle(8888)

In [7]:
type(ds_train)

tensorflow.python.data.ops.dataset_ops.ShuffleDataset

In [8]:
ds_train = ds_train.map(lambda x, y: (tf.image.random_flip_left_right(x), y))   # 隨機水平翻轉影像，加強資料差異, 但標籤 y 不變

In [9]:
ds_train = ds_train.repeat()  #無限重複

In [15]:
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size).shuffle(8888)
ds_test = ds_test.repeat()

In [10]:
model = tf.keras.models.Sequential([
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation=tf.nn.relu),
 tf.keras.layers.Dropout(0.3),
 tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax)
])

In [11]:
optimizer = tf.keras.optimizers.Adam()

In [12]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [13]:
model.fit(ds_train, epochs=epochs )

ValueError: When passing an infinitely repeating dataset, you must specify the `steps_per_epoch` argument.

In [14]:
steps_per_epoch = len(x_train) // batch_size

model.fit(ds_train, epochs=epochs, steps_per_epoch=steps_per_epoch)

Epoch 1/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0026 - accuracy: 0.1002
Epoch 2/10
1875/1875 [==============================] - 21s 11ms/step - loss: 8.3612e-07 - accuracy: 0.0998
Epoch 3/10
1875/1875 [==============================] - 22s 12ms/step - loss: 2.5427e-07 - accuracy: 0.0999
Epoch 4/10
1875/1875 [==============================] - 24s 13ms/step - loss: 8.5586e-08 - accuracy: 0.0998
Epoch 5/10
1875/1875 [==============================] - 23s 12ms/step - loss: 6.0206e-08 - accuracy: 0.0982
Epoch 6/10
1875/1875 [==============================] - 18s 9ms/step - loss: 1.0570e-08 - accuracy: 0.0998
Epoch 7/10
1875/1875 [==============================] - 16s 9ms/step - loss: 8.8390e-09 - accuracy: 0.0992
Epoch 8/10
1875/1875 [==============================] - 17s 9ms/step - loss: 4.1841e-09 - accuracy: 0.1001
Epoch 9/10
1875/1875 [==============================] - 17s 9ms/step - loss: 1.0709e-09 - accuracy: 0.0992
Epoch 10/10
1875/1875 [=============

In [16]:
model.evaluate(ds_test, steps=10)

10/10 [==============================] - 0s 32ms/step - loss: 0.0000e+00 - accuracy: 0.1156


[0.0, 0.115625]